In [1]:
%%pyspark 

from pyspark.sql.functions import explode
from pyspark.sql.functions import col, concat_ws


filewqual = "esgupload/*"

### you need to change the path to point the containing folder of the esgupload folder. this folder must contain only
### file files pushed from the power automate ESG extract flow
thefile = "abfss://workspace@stgbvnews.dfs.core.windows.net/synapse/workspaces/wsbvnews/warehouse/" + filewqual + ".json"
df_raw = spark.read.load(thefile, format='json')


### flatten the phrases and referenced orgs
ro = df_raw.withColumn("phrases", concat_ws(",", col("phrases")))\
                        .withColumn("referencedOrgs", concat_ws("|", "referencedOrgs.name"))


### explode the signals and themes
signals_ex = ro.select(ro.title,ro.excerpt, ro.phrases, ro.themes, ro.sentiment, ro.significance, ro.referencedOrgs, explode(ro.signals).alias("signal"))
df_final  = signals_ex.select(signals_ex.signal,
    signals_ex.title,signals_ex.excerpt, signals_ex.phrases,  signals_ex.sentiment, signals_ex.significance, signals_ex.referencedOrgs, explode(signals_ex.themes).alias("theme"))


spark.sql("CREATE DATABASE IF NOT EXISTS bitvorenews")


### overwrite the  table
df_final.write.mode("overwrite").saveAsTable("bitvorenews.sigtheme_ex")

df_final.printSchema()
display(df_final.limit(100))

StatementMeta(bitvorespark, 15, 1, Submitted, Running)

In [2]:
%%spark
val sql_write_df = spark.sql("SELECT title, significance, referencedOrgs, signal, theme, phrases, sentiment FROM bitvorenews.sigtheme_ex ")

sql_write_df.write.mode("overwrite").sqlanalytics("bitvoresql.dbo.sigtheme_all", Constants.INTERNAL)

StatementMeta(bitvorespark, 0, 3, Finished, Available)

sql_write_df: org.apache.spark.sql.DataFrame = [title: string, significance: string ... 5 more fields]
